<a href="https://colab.research.google.com/github/MicheleGiambelli/PlotTwisters-Project/blob/main/Transformer_funzionante_emma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import pandas as pd
import numpy as np
import string
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

!pip install transformers datasets evaluate
!pip install transformers datasets evaluate seqeval

import os


from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification, pipeline
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from datasets import Dataset, DatasetDict
import evaluate
from sklearn.model_selection import train_test_split

import spacy
from collections import Counter, defaultdict
from wordcloud import WordCloud, STOPWORDS
import plotly.express as px
import networkx as nx
from PIL import Image
import random
from itertools import combinations
from scipy.interpolate import make_interp_spline
from numpy.random import seed



In [37]:
# df1 = pd.read_csv('sample_data/file5.txt', header=None, delimiter='\t', names=['token', 'ner_tag'], skip_blank_lines=False)
# df1["token"] = df1["token"].fillna("end")
# df1["ner_tag"] = df1["ner_tag"].fillna("end")

# df2 = pd.read_csv('sample_data/file6.txt', header=None, delimiter='\t', names=['token', 'ner_tag'], skip_blank_lines=False)
# df2["token"] = df2["token"].fillna("end")
# df2["ner_tag"] = df2["ner_tag"].fillna("end")


def file_to_dataframe(file_path):
    data = []
    with open(file_path, 'r') as file:
        lines = file.readlines()  # Leggi tutte le righe del file
        for i, line in enumerate(lines):
            line = line.strip()  # Rimuove spazi e newline extra
            if not line:  # Linea vuota, segna come fine frase
                data.append(("end", "end"))
                continue

            # Controlla se la riga contiene '\t' per separare token e tag
            if '\t' in line:
                parts = line.split('\t')
                if len(parts) == 2:  # La riga è valida con 2 parti
                    token, ner_tag = parts
                    data.append((token, ner_tag))
                else:  # Riga malformata
                    print(f"Riga malformata alla linea {i + 1}: {line}")
            else:  # Riga senza '\t'
                print(f"Riga senza tabulazione alla linea {i + 1}: {line}")
                # data.append(("end", "end"))

    return pd.DataFrame(data, columns=['token', 'ner_tag'])

# Converti entrambi i file in DataFrame
df1 = file_to_dataframe('sample_data/file5.txt')
df2 = file_to_dataframe('sample_data/file5.txt')


#Assegna id di frase a file6
sentence_id = 0
for idx, row in df1.iterrows():
  df1.loc[idx, 'id'] = sentence_id
  if row['token'] == 'end' and row["ner_tag"]=="end":
    sentence_id += 1

for idx, row in df2.iterrows():
  df2.loc[idx, 'id'] = sentence_id
  if row['token'] == 'end' and row["ner_tag"]=="end":
    sentence_id += 1


df1 = df1[df1["ner_tag"] != "end"]
df2 = df2[df2["ner_tag"] != "end"]

combined_df = pd.concat([df1, df2], ignore_index=True)

# Convertire la colonna id da float a int
combined_df['id'] = combined_df['id'].astype(int)


combined_df = combined_df.dropna(subset=['token']).reset_index(drop=True) # con questo medodo possiamo rimuovere le righe del dataframe il cui valore è mancante

# Verifichiamo la presenza di righe vuote (serve più avanti per un problema al codice altrimenti)
invalid_tokens = combined_df[~combined_df['token'].apply(lambda x: isinstance(x, str))]

# Eliminare le righe con valori non stringa nella colonna 'token'
combined_df = combined_df[combined_df['token'].apply(lambda x: isinstance(x, str))].reset_index(drop=True)

In [38]:
def file_to_dataframe(file_path):
    data = []
    with open(file_path, 'r') as file:
        lines = file.readlines()  # Leggi tutte le righe del file
        for i, line in enumerate(lines):
            line = line.strip()  # Rimuove spazi e newline extra
            if not line:  # Linea vuota, segna come fine frase
                data.append(("end", "end"))
                continue

            # Controlla se la riga contiene '\t' per separare token e tag
            if '\t' in line:
                parts = line.split('\t')
                if len(parts) == 2:  # La riga è valida con 2 parti
                    token, ner_tag = parts
                    data.append((token, ner_tag))
                else:  # Riga malformata
                    print(f"Riga malformata alla linea {i + 1}: {line}")
            else:  # Riga senza '\t'
                print(f"Riga senza tabulazione alla linea {i + 1}: {line}")
                data.append(("end", "end"))

    return pd.DataFrame(data, columns=['token', 'ner_tag'])



In [39]:
# Combinare i dataset
combined_df = pd.concat([df1, df2]).reset_index(drop=True)

# Caricare il modello di lingua inglese
nlp = spacy.load('en_core_web_sm')
stopwords = nlp.Defaults.stop_words
# Punteggiatura da rimuovere
punctuation = set(string.punctuation) - {'@'}

# Funzione per eliminare stopwords e punteggiatura
def remove_stopwords_and_punctuation(df):
    df['token_cleaned'] = df['token'].apply(lambda x: x if x.lower() not in stopwords and x not in punctuation else '')
    df = df[df['token_cleaned'] != '']  # Rimuove le righe con token vuoti
    return df.drop(columns=['token']).rename(columns={'token_cleaned': 'token'})

# Applicare la funzione al dataset combinato
combined_df = remove_stopwords_and_punctuation(combined_df)

# Elimina le righe in cui il token è "rt"
combined_df = combined_df[combined_df['token'] != 'rt']

# Reset dell'indice per un DataFrame pulito
combined_df.reset_index(drop=True, inplace=True)

# Rimuovere eventuali NaN o valori non stringa
combined_df = combined_df.dropna(subset=['token']).reset_index(drop=True)
combined_df = combined_df[combined_df['token'].apply(lambda x: isinstance(x, str))].reset_index(drop=True)

# Dividere il dataset in train, validation e test
unique_ids = combined_df['id'].unique()
seed(42)
shuffled_ids = pd.Series(unique_ids).sample(frac=1, random_state=42).values
train_ids, temp_ids = train_test_split(shuffled_ids, test_size=0.3, random_state=42)
val_ids, test_ids = train_test_split(temp_ids, test_size=1/3, random_state=42)
train_df = combined_df[combined_df['id'].isin(train_ids)]
val_df = combined_df[combined_df['id'].isin(val_ids)]
test_df = combined_df[combined_df['id'].isin(test_ids)]

# Mappare le etichette NER a ID unici
all_df = pd.concat([train_df, val_df, test_df])
unique_tags = all_df['ner_tag'].unique().tolist()
tag2id = {tag: idx for idx, tag in enumerate(unique_tags)}
id2tag = {idx: tag for tag, idx in tag2id.items()}
train_df['ner_tag_id'] = train_df['ner_tag'].map(tag2id)
val_df['ner_tag_id'] = val_df['ner_tag'].map(tag2id)
test_df['ner_tag_id'] = test_df['ner_tag'].map(tag2id)

save_directory = r'C:\Users\capel\OneDrive\Desktop\Data Visualization and Text Mining\Assignment\best_ner_model'

# Crea la directory se non esiste
if not os.path.exists(save_directory):
    os.makedirs(save_directory)


# ***distilbert-base-cased***

Di seguito ho utilizzato un modello già attestrato chiamato ***distilbert-base-cased***.

Cos'è *distilbert-base-cased*:

*DistilBERT* è una versione più leggera e veloce di *BERT* (Bidirectional Encoder Representations from Transformers).                                      
Creato da *Hugging Face*, *DistilBERT* è stato ottenuto attraverso una tecnica chiamata distillazione del modello, che permette di comprimere un modello più grande (come BERT) in uno più piccolo mantenendo gran parte delle sue prestazioni.                                                                     
Il modello distilbert-base-cased è la versione di DistilBERT che distingue tra maiuscole e minuscole (case-sensitive). Ciò significa che tratta le parole "Apple" e "apple" come token diversi.
Caratteristiche principali:

                                                                                     
Dimensioni ridotte:

DistilBERT ha circa il 40% in meno di parametri rispetto a BERT base.
Questo lo rende più leggero e più veloce da addestrare e inferire.


Prestazioni:

Nonostante sia più piccolo, DistilBERT mantiene circa il 97% delle prestazioni di BERT su diversi benchmark NLP.


Vantaggi:

- Efficienza computazionale: Richiede meno memoria e risorse computazionali.
- Velocità: Più veloce sia durante l'addestramento che durante l'inferenza.
- Adatto per ambienti con risorse limitate: Ideale per applicazioni che necessitano di modelli leggeri.


Quando utilizzare distilbert-base-cased:

Quando hai limitazioni di risorse (RAM, GPU).
Se hai bisogno di addestrare o eseguire inferenze rapidamente.
Quando il leggero calo di prestazioni rispetto a BERT è accettabile per il tuo caso d'uso.


Link utili:

Modello su Hugging Face: Puoi trovare più informazioni su distilbert-base-cased sulla pagina ufficiale del modello su Hugging Face: https://huggingface.co/distilbert-base-cased


Paper originale: Se sei interessato alla tecnica di distillazione, puoi leggere il paper originale: "DistilBERT, a distilled version of BERT: smaller, faster, cheaper and lighter"

In [40]:
# Disabilitare wandb per evitare richieste di API key
os.environ["WANDB_DISABLED"] = "true"   # Disabilitare wandb (Weights and Biases): evita richieste di autenticazione a Weights and Biases, uno strumento di monitoraggio delle esperienze di machine learning.


save_directory = r'C:\Users\capel\OneDrive\Desktop\Data Visualization and Text Mining\Assignment'

# Creare la directory se non esiste
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

# **Passo 1: Mappare le etichette NER a ID unici**


all_df = pd.concat([train_df, val_df, test_df])
# Creare mapping tra tag NER e ID
unique_tags = all_df['ner_tag'].unique().tolist()
tag2id = {tag: idx for idx, tag in enumerate(unique_tags)}
id2tag = {idx: tag for tag, idx in tag2id.items()}

# Convertire 'ner_tag' in 'ner_tag_id' per ciascun DataFrame
train_df['ner_tag_id'] = train_df['ner_tag'].map(tag2id)
val_df['ner_tag_id'] = val_df['ner_tag'].map(tag2id)
test_df['ner_tag_id'] = test_df['ner_tag'].map(tag2id)



# **Passo 2: Preparare i dati per Hugging Face**

# Raggruppare i token e le etichette per frase utilizzando 'id' come identificatore
def group_data(df):
    return df.groupby('id').agg({'token': list, 'ner_tag_id': list}).reset_index()  # raggruppa i token e le etichette per frase utilizzando id come identificatore unico

train_dataset = group_data(train_df)
val_dataset = group_data(val_df)
test_dataset = group_data(test_df)



# **Passo 3: Verificare l'allineamento tra token e etichette**

def check_token_label_alignment(df_grouped):
    misaligned = df_grouped[
        df_grouped['token'].str.len() != df_grouped['ner_tag_id'].str.len()
    ]
    if not misaligned.empty:
        print("Frasi con disallineamento tra token e etichette:")
        print(misaligned)
    return misaligned.empty

assert check_token_label_alignment(train_dataset), "Disallineamento tra token e etichette nel train set"
assert check_token_label_alignment(val_dataset), "Disallineamento tra token e etichette nel validation set"
assert check_token_label_alignment(test_dataset), "Disallineamento tra token e etichette nel test set"




# **Passo 4: Creare i dataset per Hugging Face**

train_dataset = Dataset.from_pandas(train_dataset)
val_dataset = Dataset.from_pandas(val_dataset)
test_dataset = Dataset.from_pandas(test_dataset)

datasets = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})




# **Passo 5: Caricare il tokenizer e il modello pre-addestrato**

model_name = "distilbert-base-cased"  # Modello leggero e case-sensitive
tokenizer = AutoTokenizer.from_pretrained(model_name)  # AutoModelForTokenClassification è il modello pre-addestrato, configurato per il task di token classification (NER).
model = AutoModelForTokenClassification.from_pretrained(
    model_name, num_labels=len(tag2id)
)




# **Passo 6: Tokenizzazione e allineamento delle etichette**

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['token'],
        is_split_into_words=True,
        truncation=True,
        padding='max_length',
        max_length=64  # Regola questo valore in base alla lunghezza media delle frasi
    )

    labels = []
    for i, label in enumerate(examples['ner_tag_id']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Token speciali
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])  # Token principali
            else:
                label_ids.append(-100)  # Sotto-token
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Applicare la tokenizzazione ai dataset
print("Tokenizzazione dei dataset in corso...")
tokenized_datasets = datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=['id', 'token', 'ner_tag_id']
)




# **Passo 7: Preparare per l'addestramento**

# Preparare il data collator
data_collator = DataCollatorForTokenClassification(tokenizer)  # Data Collator: gestisce il padding dinamico durante la creazione dei batch, in modo che ogni batch sia della stessa lunghezza del token più lungo presente. Questo aumenta l'efficienza e la gestione della memoria.

# Definire la funzione per calcolare le metriche
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2tag[pred] for (pred, label_id) in zip(prediction, label) if label_id != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2tag[label_id] for (pred, label_id) in zip(prediction, label) if label_id != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # Utilizzare seqeval per calcolare le metriche
    seqeval = evaluate.load("seqeval")
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results.get("overall_precision", 0.0),
        "recall": results.get("overall_recall", 0.0),
        "f1": results.get("overall_f1", 0.0),
        "accuracy": results.get("overall_accuracy", 0.0),
    }






# **Passo 8: Impostare i parametri di addestramento**

training_args = TrainingArguments(
    output_dir=save_directory,  # Usa la tua directory di salvataggio qui
    num_train_epochs=3,  # Mantieni invariato il numero di epoche
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=50,
    load_best_model_at_end=True,
    report_to="none",
    fp16=torch.cuda.is_available(),
)





# **Passo 9: Creare il trainer**

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],  # TRAINING SET UTILIZZATO QUI
    eval_dataset=tokenized_datasets['validation'], # VALIDATION SET UTILIZZATO QUI
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)



# **Passo 10: Avviare l'addestramento**

print("Inizio dell'addestramento...")
trainer.train()





Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tokenizzazione dei dataset in corso...


Map:   0%|          | 0/2993 [00:00<?, ? examples/s]

Map:   0%|          | 0/855 [00:00<?, ? examples/s]

Map:   0%|          | 0/428 [00:00<?, ? examples/s]

Inizio dell'addestramento...


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.672800,0.207359,0.802809,0.765399,0.783658,0.930994
2,0.157300,0.136910,0.867406,0.879486,0.873404,0.958503
3,0.078600,0.121626,0.889655,0.898232,0.893923,0.965031


TrainOutput(global_step=282, training_loss=0.23686940365649284, metrics={'train_runtime': 2934.102, 'train_samples_per_second': 3.06, 'train_steps_per_second': 0.096, 'total_flos': 146655030462336.0, 'train_loss': 0.23686940365649284, 'epoch': 3.0})

# **Metriche per Epoca durante l'Addestramento:**

Nella tabella, possiamo vedere il progresso del modello in termini di perdita ("loss") e metriche di valutazione (Precision, Recall, F1, e Accuracy) durante le tre epoche di addestramento. Ogni colonna rappresenta il valore calcolato per una specifica epoca:

***Training Loss e Validation Loss:***

La Training Loss e la Validation Loss diminuiscono progressivamente per ciascuna epoca. Questo indica che il modello sta migliorando la sua capacità di apprendere i pattern dai dati, riducendo sia l'errore sui dati di training che l'errore sui dati di validazione. Una diminuzione consistente della Validation Loss suggerisce che il modello non sta overfittando, ma sta generalizzando bene su dati non visti.
Notiamo che la Validation Loss scende da 0.310100 nella prima epoca a 0.262881 nella terza epoca, suggerendo un miglioramento continuo nella capacità del modello di fare previsioni accurate sui dati di validazione.


***Precision, Recall, F1, e Accuracy:***

Precision, Recall, e F1 Score sono metriche fondamentali per valutare le performance del modello nel riconoscimento delle entità nominate (NER).

Precision: Rappresenta la percentuale di previsioni corrette tra tutte le previsioni fatte. Vediamo che la precision migliora da 0.679 nella prima epoca a 0.722 nella terza epoca.

Recall: Indica la percentuale di entità corrette che il modello è stato capace di individuare. Anche il recall aumenta da 0.673 a 0.755, indicando che il modello è stato in grado di riconoscere più entità correttamente man mano che l'addestramento progrediva.

F1 Score: La media armonica di precision e recall, che rappresenta un buon compromesso tra le due metriche. Anche l'F1 score aumenta da 0.676 a 0.738.

Accuracy: La Accuracy complessiva sale da 0.899 nella prima epoca a 0.918 nella terza epoca, mostrando un incremento costante nell'accuratezza del modello.

In generale, possiamo notare che i valori di tutte le metriche migliorano durante le tre epoche, indicando che il modello è stato in grado di apprendere e migliorare efficacemente.

In [41]:
# **Passo 11: Valutare il modello sui vari set**

# Valutare il modello sul set di validazione
print("Valutazione del modello sul set di validazione in corso...")
validation_metrics = trainer.evaluate(tokenized_datasets['validation'])
print("Metriche del set di validazione:")
print(validation_metrics)

# Valutare il modello sul set di test
print("Valutazione del modello sul set di test in corso...")
test_metrics = trainer.evaluate(tokenized_datasets['test'])
print("Metriche del set di test:")
print(test_metrics)



# Raccolta delle metriche
data = {
    "Set": ["Validation", "Test"],
    "Loss": [validation_metrics['eval_loss'], test_metrics['eval_loss']],
    "Precision": [validation_metrics.get('eval_precision', 0.0), test_metrics.get('eval_precision', 0.0)],
    "Recall": [validation_metrics.get('eval_recall', 0.0), test_metrics.get('eval_recall', 0.0)],
    "F1": [validation_metrics.get('eval_f1', 0.0), test_metrics.get('eval_f1', 0.0)],
    "Accuracy": [validation_metrics.get('eval_accuracy', 0.0), test_metrics.get('eval_accuracy', 0.0)]
}

# Creazione di un DataFrame Pandas
metrics_df = pd.DataFrame(data)

# Stampa della tabella riassuntiva delle metriche
print("\nTabella riassuntiva delle metriche per Validation e Test:")
print(metrics_df)





# **Passo 12: Salvare il modello addestrato**

trainer.save_model(save_directory)
tokenizer.save_pretrained(save_directory)



# **Passo 13: Funzione per la predizione su una nuova frase**

def predict_ner(sentence):
    # Tokenizzare la frase
    inputs = tokenizer.encode_plus(
        sentence,
        return_tensors='pt',
        is_split_into_words=False,
        truncation=True,
        max_length=64  # Assicurarsi che corrisponda a max_length usato in addestramento
    )
    input_ids = inputs['input_ids']

    # Mettere il modello in modalità valutazione
    model.eval()

    # Ottenere le predizioni
    with torch.no_grad():
        outputs = model(input_ids)
    predictions = torch.argmax(outputs.logits, dim=2)[0].numpy()

    # Convertire input_ids in token
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

    # Ricostruire le parole dai sub-token e assegnare le etichette
    words = []
    labels = []
    for idx, (token, pred) in enumerate(zip(tokens, predictions)):
        if token in [tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token]:
            continue  # Ignora token speciali
        if token.startswith('##'):
            words[-1] += token[2:]
        else:
            words.append(token)
            labels.append(id2tag[pred])

    # Stampare le parole e le relative etichette
    print("\nPredizione:")
    print("Parola\tEtichetta")
    for word, label in zip(words, labels):
        print(f"{word}\t{label}")


Valutazione del modello sul set di validazione in corso...


Metriche del set di validazione:
{'eval_loss': 0.12162584811449051, 'eval_precision': 0.8896551724137931, 'eval_recall': 0.8982324584895555, 'eval_f1': 0.8939232409381662, 'eval_accuracy': 0.9650308893810468, 'eval_runtime': 95.1234, 'eval_samples_per_second': 8.988, 'eval_steps_per_second': 0.284, 'epoch': 3.0}
Valutazione del modello sul set di test in corso...
Metriche del set di test:
{'eval_loss': 0.12932206690311432, 'eval_precision': 0.8754134509371555, 'eval_recall': 0.8861607142857143, 'eval_f1': 0.8807542983915697, 'eval_accuracy': 0.962435837610826, 'eval_runtime': 39.7975, 'eval_samples_per_second': 10.754, 'eval_steps_per_second': 0.352, 'epoch': 3.0}

Tabella riassuntiva delle metriche per Validation e Test:
          Set      Loss  Precision    Recall        F1  Accuracy
0  Validation  0.121626   0.889655  0.898232  0.893923  0.965031
1        Test  0.129322   0.875413  0.886161  0.880754  0.962436


In [48]:
from google.colab import drive
drive.mount('/content/drive')

# Definisci il percorso di salvataggio nel tuo Google Drive
save_directory = '/content/drive/MyDrive/Assignment'

# Crea la directory se non esiste già
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

# Salva il modello addestrato
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)
print(f"Modello e tokenizer salvati correttamente nella directory: {save_directory}")

# Non è necessario comprimere e spostare il file, poiché è già nel tuo Google Drive


Mounted at /content/drive
Modello e tokenizer salvati correttamente nella directory: /content/drive/MyDrive/Assignment


# **Tabella Riassuntiva delle Metriche per Validation e Test**

Nella tabella, vengono rappresentate delle metriche calcolate sui set di validazione e di test, dopo il completamento delle tre epoche di addestramento. Questo tipo di analisi aiuta a comprendere le prestazioni del modello su dati non visti, e quindi a valutare la sua capacità di generalizzare:

***Validation Set:***

Le metriche riportate per il set di validazione sono le seguenti:
- Loss: 0.262881, che rappresenta la misura dell'errore del modello sul set di validazione.
- Precision: 0.722178.
- Recall: 0.755444.
- F1 Score: 0.738436.
- Accuracy: 0.918093.

Questi valori suggeriscono che il modello è stato in grado di generalizzare piuttosto bene, con una buona precision e un buon recall.


***Test Set:***

Le metriche calcolate sul set di test sono:

- Loss: 0.278646, leggermente più alta rispetto al validation set, il che è normale poiché i dati di test sono completamente nuovi per il modello.
- Precision: 0.740299. Questo valore di precision è più alto rispetto a quello del set di validazione, indicando che il modello sta facendo previsioni accurate su entità anche nei dati di test.
- Recall: 0.759571, mostrando che il modello riesce a trovare un buon numero di entità corrette nei dati di test.
- F1 Score: 0.749811. Anche l'F1 score è più alto rispetto al set di validazione, suggerendo una buona capacità di compromesso tra precision e recall sui dati di test.
- Accuracy: 0.914286, un valore simile a quello del validation set, il che indica che il modello sta generalizzando bene.

Interpretazione Complessiva:
- In generale, osserviamo che il modello mostra buoni risultati sia sui dati di validazione che di test. L'accuratezza e l'F1 score relativamente alti suggeriscono che il modello ha appreso a riconoscere le entità nominate in maniera affidabile e che il rischio di overfitting è basso.
- Il Recall relativamente alto rispetto alla Precision suggerisce che il modello è piuttosto "inclusivo", cercando di identificare la maggior parte delle entità, anche se questo significa che occasionalmente potrebbe fare delle previsioni meno precise (ovvero, predire un'entità dove non c'è).
- La coerenza tra le metriche di validazione e di test è un buon segnale, indicando che il modello ha una capacità di generalizzazione sufficiente su dati mai visti prima, senza aver appreso pattern troppo specifici del training set.

In [44]:

def predict_ner(sentence):
    # Tokenizzare la frase ottenendo anche gli offset mapping
    inputs = tokenizer.encode_plus(sentence, return_tensors="pt", return_offsets_mapping=True, is_split_into_words=False)
    input_ids = inputs["input_ids"]
    offsets = inputs["offset_mapping"][0]

    # Ottenere le predizioni dal modello
    with torch.no_grad():
        outputs = model(input_ids).logits
    predictions = torch.argmax(outputs, dim=2)[0].numpy()

    # Convertire gli input_ids in token
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

    # Ricostruire le parole dai sub-token
    words = []
    labels = []
    current_word = ""
    current_label = None
    for idx, token in enumerate(tokens):
        if token == tokenizer.cls_token or token == tokenizer.sep_token:
            continue  # Ignora i token speciali [CLS] e [SEP]
        if token.startswith("##"):
            current_word += token[2:]  # Aggiungi il sub-token alla parola corrente
        else:
            if current_word != "":
                words.append(current_word)
                labels.append(current_label)
            current_word = token
            current_label = id2tag[predictions[idx]]
    # Aggiungere l'ultima parola
    if current_word != "":
        words.append(current_word)
        labels.append(current_label)

    # Stampare le parole e le relative etichette
    print("Parola\tEtichetta")
    for word, label in zip(words, labels):
        print(f"{word}\t{label}")


print("\nEsempio di predizione:")
test_sentence = "let's see if the predictions of @Plottwisters are correct and look if Emma has done a great job here in milan."
predict_ner(test_sentence)


Esempio di predizione:
Parola	Etichetta
let	O
'	O
s	O
see	O
if	O
the	O
predictions	O
of	O
@	B-PER
Plottwisters	B-PER
are	O
correct	O
and	O
look	O
if	O
Emma	B-PER
has	O
done	O
a	O
great	O
job	O
here	O
in	O
milan	B-LOC
.	O


In [45]:
test_sentence = "Hi Emma in Milan, !!! America, Obama, Trump and NATO @Plottwister is predicting pretty well."
predict_ner(test_sentence)

Parola	Etichetta
Hi	O
Emma	B-PER
in	O
Milan	B-LOC
,	O
!	O
!	O
!	O
America	B-PER
,	O
Obama	B-PER
,	O
Trump	B-PER
and	O
NATO	B-PER
@	B-PER
Plottwister	B-PER
is	O
predicting	O
pretty	O
well	O
.	O
